In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pickle

from bs4 import BeautifulSoup
import time
from datetime import date
import csv

In [24]:
import undetected_chromedriver as uc
import time
import random

# Create undetected Chrome driver
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

# Initialize the driver
driver = uc.Chrome(options=options)

# Open LinkedIn
driver.get("https://www.linkedin.com")

# Add a delay to mimic human behavior
time.sleep(random.uniform(5, 10))



In [26]:
# Save cookies to a file
with open("linkedin_cookies.pkl", "wb") as file:
    pickle.dump(driver.get_cookies(), file)

In [28]:
# Load cookies from the file
with open("linkedin_cookies.pkl", "rb") as file:
    cookies = pickle.load(file)
    for cookie in cookies:
        driver.add_cookie(cookie)

driver.refresh()  # Refresh to apply the cookies


In [30]:
url = "https://www.linkedin.com/jobs/search/?currentJobId=4191530756&f_TPR=r604800&geoId=100212432&keywords=developer&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true"
driver.get(url)
driver.implicitly_wait(10)


In [40]:
def scroll_to_bottom(driver):
    try:
        # Target the scrollable div using XPath
        scrollable_div = driver.find_element(By.XPATH, '//div[contains(@class, "LEGyEAbJtTxBjuqymLIMoGoeKJSbdDzRu")]')
        
        last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

        while True:
            # Incrementally scroll down to simulate natural scrolling
            for _ in range(20):  # 10 small scrolls
                driver.execute_script("arguments[0].scrollTop += arguments[0].clientHeight / 5", scrollable_div)
                time.sleep(1)  # Small delay to simulate natural scrolling

            new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
            if new_height == last_height:
                break  # Stop if height doesn't change (indicating no new content)
            last_height = new_height
            
    except Exception as e:
        print(f"Error while scrolling: {e}")


In [48]:
# Iterate over the item list and scrapp the needed informations
import re

Job_titles = []
Hiring_companies = []
Locations = []
Job_types = []
Job_links = []
Date_extracted = []

def scrape_page():

    driver.current_url

    # scroll the bottom of the entire page    
    scroll_to_bottom(driver)
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    Job_List = soup.find_all('div', class_="flex-grow-1 artdeco-entity-lockup__content ember-view")
    print(len(Job_List))
    
    for i in Job_List:
        
         # Extract job title
        Job_title = i.find('span')
        Job_title = Job_title.get_text(strip=True)
        Job_titles.append(Job_title if Job_title else "N/A")

         # Extract hiring company
        Hiring_company = i.find('span', class_="BvIOoxzPrVErOhCCyMrjTYTzSvdjyToic")
        Hiring_company = Hiring_company.get_text(strip=True)
        Hiring_companies.append(Hiring_company if Hiring_company else "N/A")

         # Extract location and job type
        location = i.find('li', class_="dManzPXdxNxScDQEGMgKnNjxlyOJt")
        if location:
            location_text = location.get_text(strip=True)
            match = re.match(r'^(.*?)\s*\((.*?)\)$', location_text)
            if match:
                Locations.append(match.group(1))   # First part before bracket
                Job_types.append(match.group(2))   # Second part inside bracket
            else:
                Locations.append(location_text)    # If no bracket is found, add full location
                Job_types.append("N/A")
        else:
            Locations.append("N/A")
            Job_types.append("N/A")

         #Extract job link
        Job_link = i.find('a', class_='job-card-container__link')
        Job_link = Job_link.get('href')
        Job_links.append(("https://www.linkedin.com" + Job_link) if Job_link else "N/A")
    

In [50]:
scrape_page()

25


In [36]:

page = 1

def go_to_next_page():
    global page
    try:
        # Wait until the current page button is available
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//button[@aria-current="true"]'))
        )
        
        # Find the current page and calculate next page number
        current_page = driver.find_element(By.XPATH, '//button[@aria-current="true"]')
        next_page_number = int(current_page.text) + 1
        
        print(f"➡️ Trying to move to Page {next_page_number}")
        
        # Wait until the next button with aria-label is clickable
        next_page = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f'//button[@aria-label="Page {next_page_number}"]'))
        )
        
        # Ensure the button is visible by scrolling
        driver.execute_script("arguments[0].scrollIntoView();", next_page)
        next_page.click()
        time.sleep(3)
        
        page += 1
        print(f"✅ Moved to Page {page}")
        return True
    
    except Exception as e:
        print(f"⚠️ No more pages or error: {e}")
        return False

In [82]:
go_to_next_page()

➡️ Trying to move to Page 2
✅ Moved to Page 2


True

In [62]:
Job_titles = []
Hiring_companies = []
Locations = []
Job_types = []
Job_links = []
Date_extracted = []

In [64]:
# Scrape the first page
scrape_page()

# Loop through and scrape data from subsequent pages

while go_to_next_page():
    scrape_page()  # Scrape data after clicking the "Next" button

# Once done, close the browser
#driver.quit()

25
➡️ Trying to move to Page 2
✅ Moved to Page 4
25
➡️ Trying to move to Page 3
✅ Moved to Page 5
34
➡️ Trying to move to Page 4
⚠️ No more pages or error: Message: 
Stacktrace:
	GetHandleVerifier [0x00375413+60691]
	GetHandleVerifier [0x00375454+60756]
	(No symbol) [0x001A0693]
	(No symbol) [0x001E8660]
	(No symbol) [0x001E89FB]
	(No symbol) [0x00230EE2]
	(No symbol) [0x0020D0A4]
	(No symbol) [0x0022E6FB]
	(No symbol) [0x0020CE56]
	(No symbol) [0x001DC5F3]
	(No symbol) [0x001DD444]
	GetHandleVerifier [0x005BBBC3+2446531]
	GetHandleVerifier [0x005B7198+2427544]
	GetHandleVerifier [0x005D2D5E+2541150]
	GetHandleVerifier [0x0038C3C5+154821]
	GetHandleVerifier [0x00392BED+181485]
	GetHandleVerifier [0x0037CC48+91464]
	GetHandleVerifier [0x0037CDF0+91888]
	GetHandleVerifier [0x00367980+4736]
	BaseThreadInitThunk [0x76186839+25]
	RtlGetFullPathName_UEx [0x7746906F+1215]
	RtlGetFullPathName_UEx [0x7746903D+1165]
	(No symbol) [0x00000000]



In [66]:
import pandas as pd

df = pd.DataFrame({"Job_titles":Job_titles,
                   "Hiring_companies":Hiring_companies,
                   "Locations":Locations,
                   "Job_types":Job_types,
                   "Job_links":Job_links})
styled_df = df.style.set_properties(**{'text-align': 'left'})
styled_df = styled_df.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

In [68]:
styled_df

,Job_titles,Hiring_companies,Locations,Job_types,Job_links
0,Night Shift Customer Support Officer,Chapa,"Addis Ababa, Addis Ababa, Ethiopia",On-site,https://www.linkedin.com/jobs/view/4198222047/?eBP=CwEAAAGV-8MpsqUa1QsX67nhB9UEmCWO3AENiiBRWuh5NqtBJ-R8c6u6nFDtd5HqORDGXX9oo-bFMNxvp4BVf5qqc80E4CEdc0RzHUf_bpw3Zm_xWYUTGMdWP1_2X1R8D3P_vvp1AM7kPek96pOW8zvH4S6SJsE4JKDTaB2HTPTR3ACG2tldOmulP9GiDkFhop111lksNe1-m6XRKY34-QK3fSkxtwkwzFldMcA8XvIKuY_VxUyMya6zqtRIDIcVl00197F7WBLF_QDOzBusS513194d2_0qtCVFMuMsIEuB9aA3cYEeY-EPLkEfrz1n-TL9Zwio6b-r_PD4C7y_wWl3TRWll-sZISa3fADtg3Z77SO74lJ8SzYVYq8cd0KFBxUW9r8-G5Rc6uRTyjE6UF0GQSHscE77SEHhD132xS_iyWkO4wjW2x9FsXLkXvMreO0VdD_Jh1R1QIeHlbmNyb54-w&refId=w%2BiKr66%2FGuOBuDA35O6hLg%3D%3D&trackingId=otuYQuys9lpHtIQxnidpTQ%3D%3D&trk=flagship3_search_srp_jobs&lipi=urn%3Ali%3Apage%3Ad_flagship3_search_srp_jobs%3BWfhbkQu9SzWj8pjiXUNRWg%3D%3D&lici=otuYQuys9lpHtIQxnidpTQ%3D%3D
1,Agrochemical Business Manager - Ethiopia,"EASTCHEM CO., LTD.","Addis Ababa, Ethiopia",On-site,https://www.linkedin.com/jobs/view/4197044613/?eBP=CwEAAAGV-8MpssxpA-2EbqCOi2-hYE_p219FJZ1BzZsD1FwtoA3TXgcqOAvVRAd7WKTJtOfaXUofBWN1Un8ZuQgc4jn3Fa_Mh8Ky3rWY3HzDm4gkSYHbaXUmnMvGvb7iC-tq9DlReZNGOts3EoH9-NGwoyJLHBlSNiEzrhN9y-oaWnc8peH1oOv5lFz4LneFPHZft5mmwwENG_FtyBSdMGQILFA-doHxOdQ2278seetfRDP1QaIDMAiGzyr-f-t4-ev67j7HZLQvbYowFmdz2mvUDnxyCp_ttC7W04wem-GR0gjbd-J06VRNsMBogG-apctMprnCKTnjW8wNE9qZrj6rZVy3vii-s2wf_3h__hGcTqjQTFgMGocPxVb8bN5PoDuSbwKi2rS3yIy4WL3n9Q_G7ajujaTqhBcy4VJH917diq7kRSc92YPweTc8VYz0pBYyY3zeI6hR2LKOjkXg0EJUFG15RoE5Jc-mSUYDTJe5fr5RIIK5Cdqmgg2MjqbQs7em4QpRvWWZiw&refId=w%2BiKr66%2FGuOBuDA35O6hLg%3D%3D&trackingId=1%2BDldjUKu7Kp%2BqBMg1P%2BiA%3D%3D&trk=flagship3_search_srp_jobs
2,Business Application Supervisor,ethioTelecom,"Addis Ababa, Addis Ababa, Ethiopia",On-site,https://www.linkedin.com/jobs/view/4199089687/?eBP=CwEAAAGV-8MpsnQhLhf_xTV-5gFmUy4lVHUhGCsYvw_NGNzgMdPnhdWj5P9_r_L0htiaes8KTl8stsTDCQTdOJtDyNNWkA2ggzdhiwVQ0fY_gp1vNK6s0-RnLKsIz2vgTCUbAvVslbNwVkKosIbu5BCL_pCiPUixVW155mP8v4OJjW6pXNEZiGOJbeW6HmigVIp3tvRWxcsCgO-h5A1MkGU5NgTyA6iYd4RMawDFj9Pi_yEBAWn1iO31l4n4XrS8QRZoztiVC7fqSZby0aYFmxXeBBgZvlrgg_T4bFIPm0FzDf-oeTNgJaneQDJZ2PbQ3L7ZdqZHtk61-H9EEyHWTKs8Wq0tCPROHkhxPlotD79HyTHXgZtFnDrFAjQTpMyAQ70RaAkcAeJc0y0uNZ9srg1JE7rWb3jk4nvUHiRtNMynpFd1qXFCCQqZ3Fy_71gdupGy6G8F1XUF4lyts-4n5pTYJDWAfK4Ej1jbeqGlfB0SYBgkGJtyXxfcdOKvj3OS81BcKQQ&refId=w%2BiKr66%2FGuOBuDA35O6hLg%3D%3D&trackingId=ygsoQlCDX5W6UtPwaZ%2Fdjg%3D%3D&trk=flagship3_search_srp_jobs
3,Database Administrator,micro1,EMEA,Remote,https://www.linkedin.com/jobs/view/4197081103/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=w%2BiKr66%2FGuOBuDA35O6hLg%3D%3D&trackingId=dijcU%2Fkm8gGr99TZ9Y304g%3D%3D&trk=flagship3_search_srp_jobs
4,Real estate Supervisor,real state,"Addis Ababa, Ethiopia",On-site,https://www.linkedin.com/jobs/view/4199547524/?eBP=CwEAAAGV-8MpsvNgCnX7yovfIOazbwn9UHDjV6OQuKh-c4ikndkS06I0yO5Vxr3qdVeBz0U6ZUbvB_CB-iW3pN-Wu_1sI4ZzKNk3vX68RBniMkvYlvLYPcs_ij6_dzfHiQVvJBC2M2Fis1qyF_BdkyMI78PgFTt3Kqe55vAwrwbXWHMjwniZK-QzpicwnHFaxUuo84H-2ln6LsC0hkNn9EWbQ1bJ6KJppFgp8o2lPGmf3ThmpGE9pmdimdL3lY1IrTKtRexjGaAFs_RdzZDvua6cr79JubGKpSa11EcNJkexCu1vNxGdkK_lHO1HeIeBAmepwM9gFpBVHQjdt2zEgMDCHCb5YZQf0hz1xSK_6ncsgNTT9ME6vArGqjIGyp_pcnUDGTehAm80tKvkHUqa2PPZuqoVblVpAR4oTfX_0RLE5jZISKY3pCrigQIW2cLFDWO-j7L4mH3fqYuJahHeYcrH3Q&refId=w%2BiKr66%2FGuOBuDA35O6hLg%3D%3D&trackingId=VGzfRLx4zUGlnon37LRSEg%3D%3D&trk=flagship3_search_srp_jobs
5,Sales Service Representative,AACTS Cloud,"Addis Ababa, Ethiopia",On-site,https://www.linkedin.com/jobs/view/3991360507/?eBP=CwEAAAGV-8Mps0XwkOcg_UoOH2EW7IXy9PF9qOnD5QT3612Q9jYoO_Da3U7nRawLKYYv3hNO0hOhX4Wqw0fIfrezoEFElgTXFQvIPim_Gv0l4gz5EL_xEd6Q_CInG1T4z8ylrPowOvESJMn_4hMGVsqR3h1wUGXB7Ny-ThdxbRaPtzCF5Rooybv4C2BGEcbOgbwfP0_0rGHcdiNOjmEelkhJ63OCl-8URvPmChfaMuWgvL-xnsG9a48yTNBUAo_upWChaFYo3z_CPxt87saVL5t8OLxG9FdIH2_knQzuHTLb1lAf_w4PH-GXGHe-1LGp9n3VBO1mUXd16I8BsR6RbDwppNj5SvHMJukK28vmBz7ojCjL2-_1umKgZmL-wKhebrSIHEmv-xXWrEMVDenANX1tKNSeU9GZq2r9SI3nwCPLkAFr56DCTfg_87Yv7VC5gPtUi35Vhiqq0O0RByiwMQM&refI

In [70]:
# Write to CSV file
df.to_csv('linkedin job posts.csv', index=False)